<a href="https://colab.research.google.com/github/PrachiKhatri22/PetClassify-Classification-Using-TensorFlow-CNN/blob/main/PetClassify_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Perform standard imports

In [ ]:
! pip install split-folders

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import splitfolders
import numpy as np
import copy
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('dark_background')
CUDA_LAUNCH_BLOCKING=1.

In [ ]:
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

GPU name:  []


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Humber Sem-2/Image Processing/Assign07

/content/drive/MyDrive/Humber Sem-2/Image Processing/Assign07


## Prepare train and test sets, loaders

In [ ]:
splitfolders.ratio("CatDog", output="cat_dog",
    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) # default values

Copying files: 4000 files [03:36, 18.44 files/s]


In [ ]:
train_datagen = ImageDataGenerator(rotation_range=90,
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5,
                                     height_shift_range=0.5,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     validation_split=0.2,
                                     rescale = 1./255)


test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
batch_size = 32

training_set = train_datagen.flow_from_directory('cat_dog/train/',
                                                 target_size = (224,224),
                                                 class_mode='binary',
                                                 shuffle = True,
                                                 batch_size = batch_size)
test_set = train_datagen.flow_from_directory('cat_dog/val/',
                                                 target_size = (224,224),
                                                 class_mode='binary',
                                                 shuffle = False,
                                                 batch_size = batch_size)


Found 3199 images belonging to 2 classes.
Found 801 images belonging to 2 classes.


##  Import ResNet-50

In [ ]:
base_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top = False) #etting include_top to False means it will allow adding input and output layers
output = base_model.output

In [ ]:
output

<KerasTensor: shape=(None, None, None, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                       

# Freeze upto 143 layers

In [ ]:
for layer in base_model.layers[:143]: layer.trainable = False

In [ ]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name,"-", layer.trainable)

0 input_1 - False
1 conv1_pad - False
2 conv1_conv - False
3 conv1_bn - False
4 conv1_relu - False
5 pool1_pad - False
6 pool1_pool - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_conv - False
11 conv2_block1_2_bn - False
12 conv2_block1_2_relu - False
13 conv2_block1_0_conv - False
14 conv2_block1_3_conv - False
15 conv2_block1_0_bn - False
16 conv2_block1_3_bn - False
17 conv2_block1_add - False
18 conv2_block1_out - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_conv - False
23 conv2_block2_2_bn - False
24 conv2_block2_2_relu - False
25 conv2_block2_3_conv - False
26 conv2_block2_3_bn - False
27 conv2_block2_add - False
28 conv2_block2_out - False
29 conv2_block3_1_conv - False
30 conv2_block3_1_bn - False
31 conv2_block3_1_relu - False
32 conv2_block3_2_conv - False
33 conv2_block3_2_bn - False
34 conv2_block3_2_relu - False
35 conv2_block3_3_conv - 

In [ ]:
model = Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D()) # For Flattening
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Training and Testing the model

In [ ]:
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9) ,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(training_set,
                              epochs = 10,
                              validation_data = test_set)

Epoch 1/10
100/100 [==============================] - 1229s 12s/step - loss: 0.7336 - accuracy: 0.5164 - val_loss: 12.0639 - val_accuracy: 0.4994
Epoch 2/10
100/100 [==============================] - 1170s 12s/step - loss: 0.7001 - accuracy: 0.5317 - val_loss: 8.2695 - val_accuracy: 0.4994
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 0.6940 - accuracy: 0.5280 

In [ ]:
trainAcc = [100 * x for x in history.history['accuracy']]
testAcc = [100 * x for x in history.history['val_accuracy']]

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(18,6))

ax[0].plot(history.history['loss'], 'g', lw = 3, label = 'Train')
ax[0].set_xlabel('Epochs', fontsize = 15)
ax[0].set_ylabel('Loss', fontsize = 15)
ax[0].legend(fontsize = 15)
ax[0].set_title('Training Loss', fontsize = 15)


ax[1].plot(trainAcc, label ='Train', lw = 3)
ax[1].plot(testAcc, label ='Test', lw = 3)
ax[1].set_xlabel('Epochs', fontsize = 15)
ax[1].set_ylabel('Accuracy (%)', fontsize = 15)
ax[1].set_ylim([10,110])
ax[1].set_title(f'Train Accuracy: {trainAcc[-1]:.2f}% \n Test Accuracy: {testAcc[-1]:.2f}%', fontsize = 15)
ax[1].legend(fontsize = 15)

plt.show()